# Main-and-Transfer Substation Test

## Create a New Substation

### Import CIMantic Graphs Library and CIM profile

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

### Create New Main-and-Transfer Substation

In [2]:
from cimbuilder.substation_builder import BreakerAndHalfSubstation

ImportError: cannot import name 'BreakerAndHalfSubstation' from 'cimbuilder.substation_builder.sectionalized_bus' (/home/singha42/CIM-Builder/cimbuilder/substation_builder/sectionalized_bus.py)

In [ ]:
SubBuilder = BreakerAndHalfSubstation(connection=connection, name="breaker_and_half_sub", base_voltage = 115000)
substation = SubBuilder.substation
print(substation)

Substation(mRID='626da779-1c04-4c28-a66a-30a3a4b0272d', aliasName=None, description=None, name='sectionalized_sub', Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=None, Measurements=[], OperatingShare=[], PSRType=None, ReportingGroup=[], ConnectivityNodes=[], TopologicalNode=[], AdditionalGroupedEquipment=[], Equipments=[], Bays=[], NamingFeeder=None, NormalEnergizedFeeder=[], NormalEnergizingFeeder=[], Region=None, VoltageLevels=[])


### Add IEEE 13 Feeder and IEEE 13 Assets Feeder to substation

In [ ]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)


In [ ]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)


In [ ]:
SubBuilder.new_feeder(feeder_number = 1, branch_number = 1, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_feeder(feeder_number = 2, branch_number = 2, feeder=assets13_feeder, feeder_network=assets13_network)

In [ ]:
SubBuilder.network.pprint(cim.Substation)
SubBuilder.network.pprint(cim.Feeder)

{
    "626da779-1c04-4c28-a66a-30a3a4b0272d": {
        "mRID": "626da779-1c04-4c28-a66a-30a3a4b0272d",
        "name": "sectionalized_sub"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "NormalEnergizingSubstation": "626da779-1c04-4c28-a66a-30a3a4b0272d"
    },
    "5B816B93-7A5F-B64C-8460-47C17D6E4B0F": {
        "mRID": "5B816B93-7A5F-B64C-8460-47C17D6E4B0F",
        "NormalEnergizingSubstation": "626da779-1c04-4c28-a66a-30a3a4b0272d"
    }
}


### Save Substation to XML File

In [ ]:
utils.write_xml(SubBuilder.network, '../test_output/sectionalized.xml')

![breaker-and-half](./breaker_and_half.png)

## Round-Trip Test: Load and Read from Database

### Delete all old entries from database and load new models

In [ ]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/singha42/.cache/pypoetry/virtualenvs/cim-builder-hXthbBAa-py3.10/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=33ms, elapsed=32ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=45ms, commitTime=1708968800010, mutationCount=6535</p\n></html\n>'

In [ ]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [ ]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/breaker_and_half.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="46"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="57"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="246" milliseconds="30"/>

### Create new NodeBreakerModel of Substation + Feeders

In [ ]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [ ]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "626da779-1c04-4c28-a66a-30a3a4b0272d": {
        "mRID": "626da779-1c04-4c28-a66a-30a3a4b0272d",
        "name": "sectionalized_sub",
        "ConnectivityNodes": "['bba79c83-a2b7-493f-881d-b07fbc201406', 'edfa2f9b-f265-4cd6-bb76-16c8aa638434', '01c905a3-661e-4d0c-9eb7-bcdfdef27057', '136dff61-9f59-4e52-8b2f-52663e7819c4', '379a595a-a89a-49fc-b06f-32b7242e3d47', '4fd39747-f069-42ae-89ae-c5d5ed2d4a5d', '522b6f3d-2f08-459e-98d8-247b00174ac0', '7ac62b62-37e4-4c38-a705-ba970c59a7d9']",
        "Equipments": "['884717e2-5474-4990-a9b9-e306ac31499e', '9fce2a72-c6fa-44e5-a844-b6c99c04e4f3', 'cc0bcc57-fbbc-4268-bd01-45cdb1fe48b3', 'd3f3421c-b78e-430b-8823-ba26c132dcf3', 'f8017bcc-2cce-4b31-b42b-4fbd06a27b0f', 'fd8a8a92-7cbe-47ba-8bc5-42e6b06ca6cf', '19d06a3a-0cda-44af-a5fa-993093266040', '30fc846a-cace-4460-8437-b4879db845e1', '3a17c63a-1d2d-4260-be61-a073bf1c9c9a', '426dae86-86e4-4962-b43f-edd2d401e84d', '4f8aea08-2952-480f-8c1e-5262beeb4463']",
        "NormalEnergizedFeeder": "['49AD

In [ ]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['0F1E28C3-6C44-4F88-B79C-2FDBCA4487B2', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '421E99BE-A834-4809-B924-84D88F634A45', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '673E896A-DCBF-4E43-9924-BEB31C5B6005', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '76D6D03C-967C-4E12-836D-A98DF3552BC7', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '8C58660F-C62C-4903-BE72-22F1255B1E62', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', '94F822E0-7130-4205-8597-B47110BBEF4B', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', 'DBFA8EFA-B397-40DA-9642-60E8DE4B3471', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '0124E881-B82D-4206-BBDF-37D585159872', '04984C4D-CC

In [ ]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


### Merge substation and feeders into a single XML file 

In [ ]:
utils.get_all_data(network)

ConnectivityNode
Terminal
Disconnector
Breaker
BusbarSection
Substation
Feeder
TransformerTank
ACLineSegment
PowerTransformer
LoadBreakSwitch
EnergyConsumer
Fuse
PowerElectronicsConnection
EnergySource
LinearShuntCompensator
Recloser
Location
BaseVoltage
LoadResponseCharacteristic
EnergyConsumerPhase
OperationalLimitSet
TopologicalNode
TapChangerControl
TransformerTankEnd
PowerTransformerEnd
RegulatingControl
WireSpacingInfo
ACLineSegmentPhase
PerLengthPhaseImpedance
OverheadWireInfo
ConcentricNeutralCableInfo
TapeShieldCableInfo
PhaseImpedanceData
WirePosition
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
BatteryUnit
PowerElectronicsConnectionPhase
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
CoordinateSystem
PositionPoint
SubGeographicalRegion
SwitchPhase
LinearShuntCompensatorPhase
TopologicalIsland
GeographicalRegion


In [ ]:
utils.write_xml(network, '../test_output/breaker_and_half_and_feeders.xml')
